In [4]:
import math, os, random
from collections import Counter
from typing import List, Tuple

import torch
from torchtext.datasets import SNLI
from torchtext.vocab import build_vocab_from_iterator

import spacy
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

print("Imports OK. torch version:", torch.__version__)

Imports OK. torch version: 2.9.0+cu128


In [5]:
def get_tokenizer():
    try:
        nlp = spacy.load("en_core_web_sm")
    except Exception as e:
        raise RuntimeError("Please install the spaCy model: python -m spacy download en_core_web_sm") from e
    def tokenizer(text: str) -> List[str]:
        if text is None:
            return []
        return [tok.text for tok in nlp.tokenizer(text)]
    return tokenizer

tokenizer = get_tokenizer()


In [6]:
# These are iterators; converting to lists so we can inspect multiple times.
print("Loading SNLI (may download)...")
train_iter = SNLI(split='train')
valid_iter = SNLI(split='validation')
test_iter  = SNLI(split='test')

train_list = list(train_iter)
valid_list = list(valid_iter)
test_list  = list(test_iter)

print(f"Loaded. Sizes -> train: {len(train_list)}, valid: {len(valid_list)}, test: {len(test_list)}")


Loading SNLI (may download)...


TypeError: TabularDataset.__init__() missing 3 required positional arguments: 'path', 'format', and 'fields'

In [15]:
import torchtext
print(torchtext.__version__)


0.6.0
